In [1]:
import numpy as np
import pandas as pd
import pickle
import math
import os
import matplotlib.pyplot as plt
import collections
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import collections
from utils import get_cableidx
###################
n_f=10
n_node=106
N=4000
#split
np.random.seed(1)
indices = np.random.permutation(N);print(indices[0])
n_training=int(N*0.6)
n_val=int(N*0.2)
n_test=N-n_training-n_val
print(n_training,n_val,n_test)
training_idx, val_idx,test_idx = indices[:n_training], indices[n_training:n_training+n_val], indices[-n_test:]
###################

def get_label():
    ##### label data
    y=pd.read_csv('data/raw_data/area.csv')
    label_area=y.values[1:,1:]
    #split
    l_training, l_val, l_test = label_area[training_idx],label_area[val_idx], label_area[test_idx]
    print(l_training.shape,l_val.shape,l_test.shape)
    #multi task
    def getting_multi(label_data,sf,mode):
        c_data=np.argmin(label_data,axis=1)
        a_data=np.min(label_data,axis=1)
        #c_data=c_data+1
        #c_data[a_data>0.999]=0;print('safe',np.sum(c_data==0))
        a_data=a_data[:,np.newaxis]

        #bce_data = np.zeros((len(c_data), 121))
        #bce_data[np.arange(len(c_data)),c_data] = 1

        if sf:
            np.savez_compressed('data/label_multitask/c_label_'+mode,c_data)
            np.savez_compressed('data/label_multitask/a_label_'+mode,a_data)
            #np.savez_compressed('data/label_multitask_bce/c_label_'+mode,bce_data)
            #np.savez_compressed('data/label_multitask_bce/a_label_'+mode,a_data)
        return c_data,a_data,_#,bce_data
    #save
    save_flag=False#True#
    c_training,a_training,bce_training=getting_multi(l_training,sf=save_flag,mode='train')
    c_val,a_val,bce_val=getting_multi(l_val,sf=save_flag,mode='val')
    c_test,a_test,bce_test=getting_multi(l_test,sf=save_flag,mode='test')

def get_mlpdata():
    ##### MLP input
    in_cable=np.array([1,4,9,11,16,21,24,29,31,36])-1
    with open("data/efo_signal_list.pkl", 'rb') as pickle_file:
        tension = pickle.load(pickle_file)
    tension=np.array(tension);print('whole tension',features.shape)
    tension=tension[1:,-1,in_cable]
    print('final tension',tension.shape)
    #split
    t_training, t_val, t_test = tension[training_idx],tension[val_idx], tension[test_idx]
    print(t_training.shape,t_val.shape,t_test.shape)
    #scaling
    sc=MinMaxScaler()
    t_training=sc.fit_transform(t_training)
    t_val=sc.transform(t_val)
    t_test=sc.transform(t_test)
    #save
    save_flag=False#True#
    if save_flag:
        data_path='data/data_mlp/'
        np.savez_compressed(data_path+'t_train',t_training)
        np.savez_compressed(data_path+'t_val',t_val)
        np.savez_compressed(data_path+'t_test',t_test)

##### GNN input
#node featrues - node coordinates (3)
node_loc=np.load('data/node_loc.npy')
features=node_loc.copy()
features=MinMaxScaler().fit_transform(features)
print('features',features.shape)

save_flag=True#False#
if save_flag:
    data_path='data/data_mpnn/'
    np.savez_compressed(data_path+'f_train',features)

#edge features - tension, distance, type (6)
with open("data/sensor_graph/adj_mx.pkl", 'rb') as pickle_file:
    adj_mx = pickle.load(pickle_file)
for i in range(106): adj_mx[i,i]=0
print(adj_mx.shape)
edge_data=[[i,j] for i,j in zip(np.where(adj_mx==1)[0],np.where(adj_mx==1)[1])];edge_data=np.array(edge_data)
edge_data=edge_data[edge_data[:,0]<edge_data[:,1]]

#distance
with open("data/sensor_graph/adj_mx_type_e.pkl", 'rb') as pickle_file:
    edge_de = pickle.load(pickle_file)#sacled [distance, E]
edge_d=edge_de[2][0]#;edge_e=edge_de[2][1]
print(collections.Counter(edge_d.flatten()))
for i in range(106): edge_d[i,i]=0

#area
with open("data/sensor_graph/adj_mx_type_a.pkl", 'rb') as pickle_file:
    edge_a = pickle.load(pickle_file)
print(collections.Counter(edge_a.flatten()))
for i in range(106): edge_a[i,i]=0

#make inputs for graph & edge type
edge_a=edge_a[edge_data[:,0],edge_data[:,1]]
edge_d=edge_d[edge_data[:,0],edge_data[:,1]]
print(collections.Counter(edge_a.flatten()))
edge_type1=(edge_a==97500.0).astype('float64')
edge_type2=(edge_a==3848.45).astype('float64')
edge_type3=(edge_a==8245.14480000001).astype('float64')
edge_type4=(edge_a==147500.0).astype('float64')
print(np.sum(edge_type1),np.sum(edge_type2),np.sum(edge_type3),np.sum(edge_type4))
edge_f=np.stack([edge_d,edge_type1,edge_type2,edge_type3,edge_type4],axis=1)
print(edge_data.shape,edge_f.shape)#(149, 106, 6) (149, 40) (185, 2) (185, 2)

#########tension data
in_cable=np.array([1,4,9,11,16,21,24,29,31,36])-1#list(range(3,40,4))#[3,7,11,15,19,23,27,31,35,39]
with open("data/efo_signal_list.pkl", 'rb') as pickle_file:
    tension = pickle.load(pickle_file)
tension=np.array(tension)#(4001, 4, 40)
tension=tension[1:,-1,in_cable] #4000,10
node_cable= (get_cableidx()[in_cable,1:])-1
edge_t=np.zeros((4000,185,1))
for i in range(4000):
    for j,(r,c) in enumerate(node_cable): 
        idx=np.where((edge_data == (r, c)).all(axis=1))
        edge_t[i,idx,0]=tension[i,j]
edge_f=np.array([edge_f]*4000)
edge_f=np.concatenate((edge_f,edge_t),axis=-1);print(edge_f.shape)#distance, E, area, self, tension
#indexing
e_training, e_val,e_test = edge_f[training_idx],edge_f[val_idx], edge_f[test_idx]
print(e_training.shape,e_val.shape,e_test.shape)
###scale
'''for i in [0,-1]:
    m=np.mean(e_training[...,i].flatten());std=np.std(e_training[...,i].flatten())
    e_training[...,i]=(e_training[...,i]-m)/std
    e_val[...,i]=(e_val[...,i]-m)/std
    e_test[...,i]=(e_test[...,i]-m)/std'''
sc=MinMaxScaler()
e_training[:,:,[0,-1]]=sc.fit_transform(e_training[:,:,[0,-1]].reshape((-1,2))).reshape((-1,185,2))
e_val[:,:,[0,-1]]=sc.transform(e_val[:,:,[0,-1]].reshape((-1,2))).reshape((-1,185,2))
e_test[:,:,[0,-1]]=sc.transform(e_test[:,:,[0,-1]].reshape((-1,2))).reshape((-1,185,2))
for i in range(6): print('10',i,np.min(e_val[...,i]),np.max(e_val[...,i]),np.mean(e_val[...,i]),np.std(e_val[...,i]))
#save
save_flag=True#False#
if save_flag:
    data_path='data/data_mpnn/edgetype/'
    np.savez_compressed(data_path+'edge_train',e_training)
    np.savez_compressed(data_path+'edge_val',e_val)
    np.savez_compressed(data_path+'edge_test',e_test)
    np.savez_compressed(data_path+'edge_data',edge_data)

200
2400 800 800
features (106, 3)
(106, 106)
Counter({0.0: 10866, 6100.0: 152, 7500.0: 82, 3000.0: 32, 62562.05078125: 16, 50520.1953125: 16, 38810.4375: 16, 27854.802734375: 16, 19005.525390625: 16, 9000.0: 16, 10000.0: 8})
Counter({0.0: 10866, 97500.0: 152, 3848.45: 80, 8245.14480000001: 78, 147500.0: 60})
Counter({97500.0: 76, 3848.45: 40, 8245.14480000001: 39, 147500.0: 30})
76.0 40.0 39.0 30.0
(185, 2) (185, 5)
(4000, 185, 6)
(2400, 185, 6) (800, 185, 6) (800, 185, 6)
10 0 0.0 0.9999999999999999 0.1784306897538798 0.26140132420255613
10 1 0.0 1.0 0.41081081081081083 0.49198098391276773
10 2 0.0 1.0 0.21621621621621623 0.4116634111277789
10 3 0.0 1.0 0.21081081081081082 0.4078843130792104
10 4 0.0 1.0 0.16216216216216217 0.36859950532394203
10 5 0.0 0.9968870836981987 0.028106870781757005 0.13030301320803117
